## Making demand patterns for epanet based on real data

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import timedelta

import os
import sys
module_path = os.path.abspath(os.path.join('./..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.state_comparator.sensor_data_preparator import load_and_prepare_sensor_data, create_epanet_pressure_df
from  src.state_comparator.comparator_functions import compare_real_data_with_simulated
import src.configfile as config

In [ ]:
def load_and_prepare_sensor_data(path_to_data_dir, sensor_files, pump_files, pump_column="flow_rate_value"):
    # Reads for all the 8 sensors and prepares it in a standard form for comparison
    if len(sensor_files) < 1 and len(pump_files) < 1:
        raise Exception("Files names list must contain at least one string to file !!")

    # official formula: 10^5 / /(1000 * 9,80655) = 10,197
    BARS_TO_METERS = 10.197
    pumps_time_column_name = "time"
    pumps_dict   = dict()

    for file_name in pump_files:
        temp_df = pd.read_csv(path_to_data_dir + file_name)
        prepared_df = pd.DataFrame()

        # converting to datetime, default values are in unix time
        prepared_df[pumps_time_column_name] = pd.to_datetime(temp_df[pumps_time_column_name], unit="s")
        # converting analog_2 to pressure in meters
        prepared_df[pump_column] = temp_df[pump_column]

        # grouping by time, because some rows could be duplicated
        pumps_dict[file_name] = prepared_df.groupby(prepared_df[pumps_time_column_name]).mean()

    for pump_name in pumps_dict:
        pumps_dict[pump_name] = pumps_dict[pump_name].resample("1H").mean()

    sensors_pumps_dict = rename_dict_keys_and_merge(pumps_dict)
    return sensors_pumps_dict

def rename_dict_keys_and_merge(pump_dict, pump_names_dict=None):
    sensor_pump_dict_new = dict()

    if pump_names_dict is None:
        pump_names_dict = dict()
        pump_names_dict["braila_flow211206H360.csv"] = "Jonctiune-3974"     # ("Apollo", "Jonctiune-3974"), ,
        pump_names_dict["braila_flow211106H360.csv"] = "Jonctiune-J-3"      # ("GA-Braila", "Jonctiune-J-3")
        pump_names_dict["braila_flow318505H498.csv"] = "Jonctiune-J-19"     # ("RaduNegru2", "Jonctiune-J-19")
        pump_names_dict["braila_flow211306H360.csv"] = "Jonctiune-2749"     # ("RaduNegruMare", "Jonctiune-2749")

    for pump in pump_dict:
        new_name = pump_names_dict[pump]
        sensor_pump_dict_new[new_name] = pump_dict[pump]

    return sensor_pump_dict_new

In [ ]:
prepared_data_dict = load_and_prepare_sensor_data("./../data/sensor_data/", config.SENSOR_FILES, config.PUMP_FILES)
"""
sensor_names_dict["braila_pressure5770.csv"] = "SenzorComunarzi-NatVech"
sensor_names_dict["braila_pressure5771.csv"] = "SenzorComunarzi-castanului"
sensor_names_dict["braila_pressure5772.csv"] = "SenzorChisinau-Titulescu"
sensor_names_dict["braila_pressure5773.csv"] = "SenzorCernauti-Sebesului"
pump_names_dict["braila_flow211206H360.csv"] = "Jonctiune-3974"     # ("Apollo", "Jonctiune-3974"), ,
pump_names_dict["braila_flow211106H360.csv"] = "Jonctiune-J-3"      # ("GA-Braila", "Jonctiune-J-3")
pump_names_dict["braila_flow318505H498.csv"] = "Jonctiune-J-19"     # ("RaduNegru2", "Jonctiune-J-19")
pump_names_dict["braila_flow211306H360.csv"] = "Jonctiune-2749"     # ("RaduNegruMare", "Jonctiune-2749")
"""
sensor_lit = "Jonctiune-3974"
display(prepared_data_dict[sensor_lit])

In [ ]:
for sensor in prepared_data_dict:
    prepared_df = prepared_data_dict[sensor]
    prepared_df = prepared_df.loc[(prepared_df.index > pd.to_datetime("2021-03-01 10:00:00") )]
    prepared_df
    demand_pattern = prepared_df.groupby(prepared_df.index.hour).mean()

    # display(demand_pattern)
    fig = go.Figure()
    fig.add_trace(go.Bar(x=demand_pattern.index, 
                    y=demand_pattern["flow_rate_value"],
                    text=sensor)
                 )
    fig.update_layout(
        title=sensor,
        xaxis_title="Hour of the day",
        yaxis_title="Flow rate value m3/h",
    )
    fig.show()
    print(demand_pattern["flow_rate_value"].to_numpy())

In [ ]:
# prepared_df[(prepared_df["flow_rate_value"] >= 60)]

## Analysis by day of the week

In [ ]:
for sensor in prepared_data_dict:
    prepared_df = prepared_data_dict[sensor]
    prepared_df = prepared_df.loc[(prepared_df.index > pd.to_datetime("2021-03-01 10:00:00") )]

    hours_of_the_week = prepared_df.groupby((prepared_df.index.dayofweek) * 24 
                                            + (prepared_df.index.hour)).mean().rename_axis('HourOfWeek')

    fig = go.Figure()
    fig.add_trace(go.Bar(x=hours_of_the_week.index, 
                    y=hours_of_the_week["flow_rate_value"],
                    text=sensor)
                 )
    fig.update_layout(
        title=sensor,
        xaxis_title="Hour of the week",
        yaxis_title="Flow rate value m3/h",
    )
    fig.show()
    # print(hours_of_the_week["flow_rate_value"].to_numpy())

In [ ]:
# ("Apollo", "Jonctiune-3974"), ,
# ("GA-Braila", "Jonctiune-J-3")
# ("RaduNegru2", "Jonctiune-J-19")
# ("RaduNegruMare", "Jonctiune-2749")
def get_day_data_for_sensor(sensor_name, day):
    dt_day = pd.to_datetime(day)
    dt_next_day = dt_day + timedelta(days=1)
    one_day_data = prepared_data_dict[sensor_name]
    one_day_data = one_day_data.loc[(one_day_data.index >= dt_day) & (one_day_data.index < dt_next_day)]

    hours_of_the_week = one_day_data # one_day_data.groupby(one_day_data.index.hour).mean()
    display(hours_of_the_week)
    
get_day_data_for_sensor("Jonctiune-2749", "2021-01-01 10:00:00")

In [ ]:
# display(demand_pattern)

## Checking patterns with cumulative flow

In [ ]:
prepared_cumulative_data_dict = load_and_prepare_sensor_data("./../data/sensor_data/", config.SENSOR_FILES, 
                                                             config.PUMP_FILES, "consumer_totalizer")

sensor_cumulative = "Jonctiune-3974"
# display(prepared_cumulative_data_dict[sensor_cumulative])